In [102]:
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium import webdriver    
import time
import pandas as pd
import re
import collections
import nltk
from nltk import word_tokenize, Text, FreqDist
from nltk.corpus import wordnet as wn
from nltk.corpus import PlaintextCorpusReader
from nltk.corpus import stopwords
wnl = nltk.WordNetLemmatizer()
import pathlib
from newspaper import Article
from urllib.parse import urlparse
# importing itertools for accumulate()
import itertools
# importing functools for reduce()
import functools
from tqdm.notebook import tqdm_notebook
import json
import praw
import seaborn as sns
import requests
from bs4 import BeautifulSoup
import os
import concurrent.futures
import time
from tqdm import tqdm
import gc
from nltk.corpus import stopwords
from IPython.display import display, clear_output
import pickle

In [103]:
def selenium_init():
    #initalise crawler option(s)
    options = webdriver.ChromeOptions()
    options.add_experimental_option("detach", True)
    #options.add_argument('headless')
    #PROXY = "11.456.448.110:8080"
    #options.add_argument('--proxy-server=%s' % PROXY)
    ROOT_DIR = pathlib.Path().absolute()
    print(ROOT_DIR)
    driver = webdriver.Chrome((str(ROOT_DIR)+"/chromedriver"), options=options)
    driver.set_window_size(1120, 1000)
    
    return driver

In [109]:
def getCommentsList(items,p_id=""):
    #print('\n-- Parent:',p_id)
    
    out = []
    
    for idx, comment_item in enumerate(items):
        #1. comment id 
        try:
            c_id = (comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@id, 'comment-')]")).get_attribute('id')
        except Exception as e:
            c_id = ""
        #print('\n\t-- DEBUG: p_id ', p_id, 'c_id ', c_id)
        
        #2. comment user
        try:
            c_user = (comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class, 'wpd-comment-author')]")).text
        except Exception as e:
            c_user = ''
        #print('\n-- DEBUG: ',c_user)
        
        #3. comment datetime
        try:
            c_datetime = (comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class,'wpd-comment-date')]")).get_property('title')
        except Exception as e:
            c_datetime = ''
        #print('\n-- DEBUG: ', c_datetime)
        
        #4. comment text
        try:
            c_text = (comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class,'wpd-comment-text')]")).text
        except Exception as e:
            c_text = ''
        #print('\n-- DEBUG: ', c_text)
        
        #5. post score
        try:
            c_score = (comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class,'wpd-vote-result wpd-vote-result-like wpd-up')]")).text
        except Exception as e:
            c_score = ''

        one_comment_item ={
                'cmt_id' : c_id,
                'cmt_replyTo' : p_id,
                'cmt_user': c_user,
                'cmt_score':c_score,
                'cmt_text': c_text,
                'cmt_datetime': c_datetime
        }

        #print(one_comment_item)

        if c_id != '' and c_user != '' and c_text != '' and c_datetime != '':
            out.append(one_comment_item)
        #6. check if has child element:
        try:
            # This comment has a child element, recursion goes on:
            # child base node
            #child_element = comment_item.find_element("xpath", ".//descendant-or-self::div[contains(@class,'child')]")
            # children nodes
            children = comment_item.find_elements("xpath","./child::div[contains(@class, 'wpd-comment wpd-reply')]")
            #print('\n\t-- DEBUG: ', len(children), children)
            if len(children) > 0:
                out.extend(getCommentsList(children, p_id=c_id))
        except Exception as e:
            # This comment does not has a child element:
            pass
    
    return out

def getCommentItems(driver, c_url, sleep=8):
    
    driver.get(c_url)
    
    wait = WebDriverWait(driver, 8)
    
    clicks = ["//div[contains(@class, 'wpd-load-more-submit-wrap')]/button[contains(@class,'wpd-load-comments wpd-prim-button')]",
              "//div[contains(@class,'wpd-toggle wpd-hidden wpd_not_clicked')]//i[contains(@class,'fas fa-chevron-down')]"]
    
    page_loaded = wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
    
    if page_loaded:
        time.sleep(sleep)
        for each in clicks:
            try:
                each_loaded = wait.until(EC.presence_of_element_located((By.XPATH, each)))
                if each_loaded:
                    expanders = driver.find_elements("xpath", each)
                    for each in expanders:
                        each.click()
                        time.sleep(sleep)
            except Exception as e:
                pass
    
        xpath_comments_dir = "//div[contains(@class, 'wpd-thread-list')]/div[contains(@id, 'wpd-comm-')]"

        root_items = driver.find_elements("xpath", xpath_comments_dir)
        
        out = getCommentsList(root_items, p_id="")
    
    return out

In [106]:
import pandas as pd

def load_pickle_object(file_path):
    with open(file_path, "rb") as input_file:
        data = pickle.load(input_file)
        return data

driver = selenium_init()

data_path = "news_crawler/output/raw/ONLINE_CITI_2021-12-31_2023-12-31.txt"

url_1 = "https://www.theonlinecitizen.com/2023/02/14/workplace-discrimination-bill-proposed-in-2021-but-expected-to-be-enacted-in-2024-as-non-native-sporeans-in-workforce-grow-to-an-estimated-55-6/"
url_2 = "https://www.theonlinecitizen.com/2023/02/14/new-jobs-skills-integrators-announced-as-measures-to-support-employment-and-job-training/"
url_3 = "https://www.theonlinecitizen.com/2023/02/06/stern-warnings-was-an-in-between-solution-as-cpib-lacks-sufficient-evidence-to-prosecute-six-former-kom-executives-for-bribery-said-indranee-rajah/"

data_df = pd.read_json(data_path)

data_df = df.rename(columns={'date_publish': 'published_datetime', 'article': 'content', 'news_title': 'title', '\news_category':'category'})

C:\Users\liux5\Documents\Project 3 - DSTA


C:\Users\liux5\AppData\Local\Temp\ipykernel_21700\2126440524.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome((str(ROOT_DIR)+"/chromedriver"), options=options)


In [110]:
all_comments = []

for idx, row in df.iterrows():
    url = row['url']
    
    print(f'\n--DEBUG: {idx} {url}')
    
    one_comments = getCommentItems(driver,url,sleep=4)
    
    all_comments.append(one_comments)



--DEBUG: 0 https://www.theonlinecitizen.com/2022/12/13/7-reasons-why-the-cannot-lower-bto-prices-narrative-is-not-so-convincing/

--DEBUG: 1 https://www.theonlinecitizen.com/2022/12/13/china-protester-detained-for-nine-days-mother/

--DEBUG: 2 https://www.theonlinecitizen.com/2022/11/14/forex-trading-tips-currency-trading-tips-for-beginners/

--DEBUG: 3 https://www.theonlinecitizen.com/2022/12/12/air-india-close-to-sealing-historic-order-for-up-to-500-jets-with-sia-needing-to-inject-more-money/

--DEBUG: 4 https://www.theonlinecitizen.com/2022/12/12/no-intergenerational-equity-in-pap-govts-fiscal-policies/

--DEBUG: 5 https://www.theonlinecitizen.com/2022/12/12/senior-minister-teo-chee-hean-and-minister-k-shanmugam-attending-world-cup-2022-in-qatar/

--DEBUG: 6 https://www.theonlinecitizen.com/2022/12/12/workers-rights-group-to-meet-watsons-over-the-right-to-sit-for-retail-workers/

--DEBUG: 7 https://www.theonlinecitizen.com/2022/12/10/temasek-backed-crypto-company-amber-group-rumore

TimeoutException: Message: 


In [111]:
import json
with open("data/theOnlineCitizen.json", "w") as outfile:
    json.dump({"data": all_comments}, outfile)

In [112]:
driver.close()

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=110.0.5481.178)
Stacktrace:
Backtrace:
	(No symbol) [0x006937D3]
	(No symbol) [0x00628B81]
	(No symbol) [0x0052B36D]
	(No symbol) [0x0051BE5A]
	(No symbol) [0x0051C63A]
	(No symbol) [0x0051C5D8]
	(No symbol) [0x0050D0BA]
	(No symbol) [0x0050DBA7]
	(No symbol) [0x0058489B]
	(No symbol) [0x0057B3E3]
	(No symbol) [0x00550D97]
	(No symbol) [0x0055253D]
	GetHandleVerifier [0x0090ABF2+2510930]
	GetHandleVerifier [0x00938EC1+2700065]
	GetHandleVerifier [0x0093C86C+2714828]
	GetHandleVerifier [0x00743480+645344]
	(No symbol) [0x00630FD2]
	(No symbol) [0x00636C68]
	(No symbol) [0x00636D4B]
	(No symbol) [0x00640D6B]
	BaseThreadInitThunk [0x75916BD9+25]
	RtlGetFullPathName_UEx [0x76F88FD2+1218]
	RtlGetFullPathName_UEx [0x76F88F9D+1165]


In [124]:
test = data_df.loc[:1023]
test['comments'] = all_comments

C:\Users\liux5\AppData\Local\Temp\ipykernel_21700\4187439535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test['comments'] = all_comments


In [164]:
from datetime import datetime
import ast

INSERT_API = 'http://10.2.56.213:8086/insert'

current_time = datetime.today().strftime("%Y-%m-%d") + 'T' + datetime.today().strftime("%H:%M")

for idx, row in test.iterrows():
    
    article_data = {'org_title': row['title'],
            'org_content': row['content'],
            'title':row['title'],
            'content': row['content'],
            'lang': 'EN',
            'category': row['category'],
            'source_id': 15,
            'published_datetime': row['published_datetime'],
            'last_modified': str(current_time)
       }
    
    trans_response = requests.post(INSERT_API,json={'table':'dsta_db.news', 'data': article_data })
    
    article_id = (ast.literal_eval(trans_response.text))['lastrowid']
    
    if len(row['comments'])>0:
        comments_df = pd.DataFrame(row['comments'])
        #display(comments_df)
        for idx, c_row in comments_df.iterrows():
            published_datetime = pd.to_datetime(c_row['cmt_datetime'], format='%d/%m/%Y %I:%M %p', errors='ignore')
            comment_data = {
            'cmt_org_content': c_row['cmt_text'],
            'cmt_content': c_row['cmt_text'],
            'cmt_id': c_row['cmt_id'],
            'cmt_user':c_row['cmt_user'],
            'cmt_replyTo':c_row['cmt_replyTo'],
            'cmt_likes': c_row['cmt_score'],
            'cmt_published_datetime': str(published_datetime),
            'translated': True,
            'lang': 'EN',
            'last_modified':str(current_time),
            'cmt_article_id': article_id,
            'source_id': 15
            }
            
            trans_response = requests.post(INSERT_API,json={'table':'dsta_db.comments', 'data': comment_data })
            print(trans_response.status_code)
            #break

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
500
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
